# recslam catalog

> Generation of catalog from recslam data


In [ ]:
# | default_exp tools.recslam.catalog

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# | export
# | hide

# basic imports
from __future__ import annotations

# sys and paths imports
import json
import logging
import os
import re
import sys
import time
import weakref

# typing imports
from abc import ABC, abstractmethod
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Generic, Literal

# cv and image imports
import cv2
import ipywidgets as widgets
import numpy as np
import pandas as pd
from dotenv import load_dotenv

# widgets imports
from IPython.display import display
from PIL import Image as PImage
from transitions import Machine

from ds_contrib.core.data.video import sample_frames_from_video
from ds_contrib.core.files.structure import FSNode, GSBrowserFileStructure
from ds_contrib.core.paths import (
    Directory,
    PathLike,
    handle_existing,
    handle_existing_path,
    list_paths,
    pathify,
)

# visualization imports
from ds_contrib.core.plotting import plot
from ds_contrib.core.utils import Iterifiable, T, listify

# from ds_contrib.tools.browser import ImageBrowser
from ds_contrib.tools.io.gscloud import GSBrowser, GSBrowserContext

In [ ]:
# | export
# | hide

logger = logging.getLogger(__name__)

In [ ]:
# | hide

CWD = Path.cwd()
REPO_DIR = Path(*CWD.parts[: CWD.parts.index("ds_contrib") + 1])
CONFIGS_DIR = REPO_DIR / "configs"
ENV_DIR = CONFIGS_DIR / "env/local"

with open(CONFIGS_DIR / "storage/gscloud/projects_vars.json") as f:
    projects = json.load(f)

# choose project
project = projects["dev"]
env_path = Path(ENV_DIR / f'{project["env"]}_roadly.env')

_ = load_dotenv(env_path)  # read local .env file
google_app_creds = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
roadly_cookie = os.getenv("ROADLY_COOKIE")
print(f"Initial configuration has finished:\nProject: {project}")

Initial configuration has finished:
Project: {'project': 'roadly-project-dev', 'env': 'dev', 'coldline_name': 'standard'}


In [ ]:
browser = GSBrowser(
    project=project["project"],
    credentials=google_app_creds,
)

## RecSLAM item

> Browse and download data from a single RecSLAM folder


In [ ]:
# | export


@dataclass
class CameraMetadataDTO:
    field_of_view: float
    camera_type: str
    fps: int
    resolution: tuple[int, int]
    focal_length: float


@dataclass
class DeviceMetadataDTO:
    os: str
    device: str
    device_raw: str
    app_version: str
    exposure_mode: str
    max_exposure_time: float
    camera_lense_position: int
    orientation: str
    wide_camera: CameraMetadataDTO
    ultra_wide_camera: CameraMetadataDTO

    @classmethod
    def from_json(cls, json_file):
        with open(json_file) as f:
            data = json.load(f)
            wide_camera = CameraMetadataDTO(
                data["cam_fov_2"],
                data["camera_type_2"],
                data["fps2"],
                tuple(map(int, data["video_resolution_2"].split("x"))),
                data["cam_fx_2"],
            )
            ultra_wide_camera = CameraMetadataDTO(
                data["cam_fov"],
                data["camera_type"],
                data["fps"],
                tuple(map(int, data["video_resolution"].split("x"))),
                data["cam_fx"],
            )
            return cls(
                data["os"],
                data["device"],
                data["device_raw"],
                data["app_version"],
                data["exposure_mode"],
                data["max_exposure"],
                data["cam_lens_pos"],
                data["orientation"],
                wide_camera,
                ultra_wide_camera,
            )

## Show catalog


Reading RecSLAM catalog from file


In [ ]:
sampled_datasets = pd.read_csv("/Users/arseniy/Downloads/выборка.csv", index_col=0)

In [ ]:
sampled_datasets.rename(columns={"Unnamed: 0": "dataset_id"}, inplace=True)
sampled_datasets.set_index("name", inplace=True)

In [ ]:
# | export


class Metadata(ABC):
    @abstractmethod
    def __getitem__(self, item) -> dict[str, Any]:
        raise NotImplementedError()


class DataFrameMetadata(Metadata):
    def __init__(self, df: pd.DataFrame, index_column=None):
        # Warning: this may slow down the init
        self._df = df.copy()
        if index_column:
            self._df.set_index(index_column, inplace=True)

    def __getitem__(self, item) -> dict[str, Any]:
        return self._df.loc[item].to_frame()

    def keys(self):
        return self._df.index.tolist()

    def _repr_html_(self):
        return self._df._repr_html_()

In [ ]:
meta = DataFrameMetadata(sampled_datasets)

In [ ]:
# | export


class MetaDataBox:
    def __init__(self, meta: Metadata) -> None:
        # self.item_meta_box = widgets.Textarea(
        #     value="",
        #     description="",
        #     disabled=True,
        #     layout=widgets.Layout(width="80%", height="30em"),
        #     placeholder="Row content will be displayed here",
        # )
        self.item_meta_box = widgets.Output()
        self._meta = meta

    def set_item_meta(self, index_id):
        items: pd.DataFrame = self._meta[index_id]
        # drop linestring column due too long values
        items = items.drop(index=["Route"])
        with pd.option_context("display.max_colwidth", None):
            with self.item_meta_box:
                self.item_meta_box.clear_output()
                display(items)

    @property
    def widget(self):
        return self.item_meta_box


class ItemsDropdownWithMeta(Generic[T]):
    def __init__(
        self, items: list[T], meta: Metadata | None = None, description=None
    ) -> None:
        self._items = items
        self._meta = meta
        self._description = description
        self._setup_gui()
        self._setup_callbacks()

    def _setup_gui(self):
        self.dropdown = widgets.Dropdown(
            options=self._items,
            description=self._description,
            disabled=False,
            layout=widgets.Layout(width="30%"),
        )
        self._metadata_box = MetaDataBox(self._meta)
        self._widget = widgets.VBox([self.dropdown, self._metadata_box.widget])

    def _setup_callbacks(self):
        self.dropdown.observe(self._cb_on_dropdown_change, names="value")

    def _cb_on_dropdown_change(self, change):
        self._metadata_box.set_item_meta(change.new)

    @property
    def widget(self):
        # use it to prefill the dropdown
        self.dropdown.notify_change(
            {"name": "value", "old": None, "new": self._items[0], "type": "change"}
        )
        return self._widget

    @property
    def value(self) -> T:
        return self.dropdown.value

In [ ]:
sampled_datasets.sort_values("iri_mean", ascending=False)

,dataset_id,Path,Datetime,Country,City,Speed,Route,iri_mean,iri_var,ride_quality,wide,ultrawide,video_data,local_path,fps,frame_count,duration,Решение,Причина,video_url
name,,,,,,,,,,,,,,,,,,,,
2022-12-17_15-33-16_83AB9828-5B0,3537,2022-12-17_15-33-16_83AB9828-5B0,2022-12-17 14:33:16,Uzbekistan,NaN,13.874303,"LINESTRING (69.074585 41.123241, 69.074628 41....",11.176429,23.879309,RideQuality.BAD,True,True,False,data/downloads/2022-12-17_15-33-16_83AB9828-5B...,5.000035,1200,239.998333,Подходит,NaN,https://storage.cloud.google.com/roadly-dev-st...
2022-07-17_09-05-31_4453D774-04F,3652,Alanya/2022-07-17_09-05-31_4453D774-04F,2022-07-17 10:05:32,Turkey,NaN,13.399114,"LINESTRING (29.113155 36.661371, 29.113136 36....",10.881955,36.128544,RideQuality.BAD,True,True,False,data/downloads/2022-07-17_09-05-31_4453D774-04...,5.000158,529,105.796667,Не подходит?,Брусчатка,https://storage.cloud.google.com/roadly-dev-st...
2022-12-17_13-32-09_83AB9828-5B0,3936,2022-12-17_13-32-09_83AB9828-5B0,2022-12-17 12:32:10,Uzbekistan,NaN,10.560251,"LINESTRING (69.0679 41.129758, 69.067904 41.12...",10.391300,16.992052,RideQuality.BAD,True,True,False,data/downloads/2022-12-17_13-32-09_83AB9828-5B...,5.000132,316,63.198333,Подходит,NaN,https://storage.cloud.google.com/roadly-dev-st...
2022-12-17_15-44-08_83AB9828-5B0,14,2022-12-17_15-44-08_83AB9828-5B0,2022-12-17 14:44:08,Uzbekistan,NaN,13.776954,"LINESTRING (69.136316 41.178227, 69.136369 41....",10.047226,18.226671,RideQuality.BAD,True,True,False,data/downloads/2022-12-17_15-44-08_83AB9828-5B...,5.000074,1129,225.796667,Подходит,NaN,https://storage.cloud.google.com/roadly-dev-st...
2022-12-17_13-28-38_83AB9828-5B0,96,2022-12-17_13-28-38_83AB9828-5B0,2022-12-17 12:28:39,Uzbekistan,NaN,7.266367,"LINESTRING (69.06429 41.132203, 69.064289 41.1...",9.327448,12.096538,RideQuality.BAD,True,True,False,data/downloads/2022-12-17_13-28-38_83AB9828-5B...,5.000085,490,97.998333,Подходит,NaN,https://storage.cloud.google.com/roadly-dev-st...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-22_21-25-36_4453D774-04F,3721,Alanya/2022-06-22_21-25-36_4453D774-04F,2022-06-22 21:25:36,Georgia,Batumi,5.429843,"LINESTRING (41.61561 41.63951, 41.61561 41.639...",3.270516,0.705873,RideQuality.GOOD,True,True,False,data/downloads/2022-06-22_21-25-36_4453D774-04...,5.000180,1391,278.190000,Не подходит,Ночь\Лужи\Пробки,https://storage.cloud.google.com/roadly-dev-st...
2022-03-08_12-05-03_4453,4081,Antalya/2022-03-08_12-05-03_4453,2022-03-08 13:05:03,Turkey,NaN,8.519116,"LINESTRING (32.093135 36.491349, 32.093135 36....",3.228163,0.367494,RideQuality.GOOD,True,True,False,data/downloads/2022-03-08_12-05-03_4453.mp4,5.000173,1444,288.790000,Подходит,NaN,https://storage.cloud.google.com/roadly-dev-st...
2022-03-08_12-10-43_4453,4062,Antalya/2022-03-08_12-10-43_4453,2022-03-08 13:10:43,Turkey,NaN,7.458303,"LINESTRING (32.110532 36.475415, 32.110578 36....",3.212623,0.672564,RideQuality.GOOD,True,True,False,data/downloads/2022-03-08_12-10-43_4453.mp4,5.000161,519,103.796667,Подходит,NaN,https://storage.cloud.google.com/roadly-dev-st...


In [ ]:
meta = DataFrameMetadata(sampled_datasets)
catalog_sampler = ItemsDropdownWithMeta(meta.keys(), meta=meta, description="Videos")
display(catalog_sampler.widget)

## Show sample


In [ ]:
chosen_session = catalog_sampler.value
video_url = sampled_datasets.loc[chosen_session, "video_url"]
session_path = f'roadly-dev-videos/{sampled_datasets.loc[chosen_session, "Path"]}/'

In [ ]:
downloads_dir = Directory(f"downloads/{chosen_session}", temporary=False)

dfs = GSBrowserFileStructure(
    browser,
    downloads_dir,
    CONFIGS_DIR / "storage/recslam/recslam_structure.json",
    session_path,
)

In [ ]:
dfs["camera_wide/video"]

,,path,description,type,exists,meta[local_path],meta[cached]
prefix,name,,,,,,
camera_wide,video,video_2,Video captured by the wide camera (main camera...,file,True,/Users/arseniy/Projects/dev/ds_contrib/nbs/cor...,False


In [ ]:
dfs.get("camera_wide/video")

[None]

In [ ]:
video_path: Path = dfs["camera_wide/video"].meta["local_path"]
frames_path = video_path.parent / "frames"
frames_dir = Directory(frames_path, temporary=False)

NameError: name 'dfs' is not defined

In [ ]:
sample_frames_from_video(video_path, frames_dir)

Sampling frames:   0%|          | 0/106 [00:00<?, ?it/s]

In [ ]:
# ib = ImageBrowser(list(list_paths(frames_path, recursive=True, sort=True)))
# ib.browse()

# Single widget


In [ ]:
# | export


class RecslamBrowserWidget:
    def __init__(self, catalog: pd.DataFrame, gs_browser: GSBrowser) -> None:
        self._catalog = catalog
        self._meta = DataFrameMetadata(catalog)
        self._frames_path = None
        self._browser = gs_browser

        self._init_gui()
        self._setup_callbacks()
        self._init_state_machine()
        self._downloads_dir = Directory(f"downloads/", temporary=True)
        self._downloads_path = self._downloads_dir.path

        self._finalizer = weakref.finalize(self, self._cleanup, self._downloads_dir)

        self.RESET()

    @classmethod
    def _cleanup(cls, downloads_dir: Directory):
        downloads_dir.cleanup()

    def _init_state_machine(self):
        states = ["selection", "downloading", "sampling", "ready"]
        self.fsm = Machine(model=self, states=states, initial="selection")
        self.fsm.add_transition(trigger="RESET", source="*", dest="selection")
        self.fsm.add_transition(
            trigger="DOWNLOADING", source="selection", dest="downloading"
        )
        self.fsm.add_transition(
            trigger="SAMPLING", source="downloading", dest="sampling"
        )
        self.fsm.add_transition(trigger="READY", source="sampling", dest="ready")

    def _init_gui(self):
        self._catalog_sampler = ItemsDropdownWithMeta(
            self._meta.keys(), meta=self._meta, description="Videos"
        )
        self._process_button = widgets.Button(
            description="Process", style={"button_color": "lightgreen"}
        )
        self._widget = widgets.VBox(
            [self._catalog_sampler.widget, self._process_button]
        )

    def _setup_callbacks(self):
        self._process_button.on_click(self._cb_on_process_button_click)
        self._catalog_sampler.dropdown.observe(
            self._cb_on_catalog_sampler_change, names="value"
        )

    def _cb_on_process_button_click(self, change=None):
        self._process_button.disabled = True
        self._catalog_sampler.dropdown.disabled = True
        self.download_video()
        self.sample_frames()
        self.READY()
        self._catalog_sampler.dropdown.disabled = False
        self._process_button.disabled = False

    def _cb_on_catalog_sampler_change(self, change):
        self.RESET()

    def download_video(self):
        self.DOWNLOADING()
        self._chosen_session = self._catalog_sampler.value
        # video_url = self._catalog.loc[chosen_session, "video_url"]
        self._session_path = (
            f'roadly-dev-videos/{self._catalog.loc[self._chosen_session, "Path"]}/'
        )

        self._session_dir = Directory(
            f"{self._downloads_path}/{self._chosen_session}", temporary=False
        )
        self._dfs = GSBrowserFileStructure(
            self._browser,
            self._session_dir,
            CONFIGS_DIR / "storage/recslam/recslam_structure.json",
            self._session_path,
        )
        self._dfs.get("camera_wide/video")

    def sample_frames(self):
        self.SAMPLING()
        self._video_path: Path = self._dfs["camera_wide/video"].meta["local_path"]
        self._frames_path = self._video_path.parent / "frames"
        self._frames_dir = Directory(self._frames_path, temporary=False)
        sample_frames_from_video(self._video_path, self._frames_dir)

    @property
    def widget(self):
        return self._widget

    @property
    def frames(self):
        return list(
            list_paths(self._frames_path, recursive=True, sort=True)
            if self._frames_path
            else []
        )

In [ ]:
recslam_browser = RecslamBrowserWidget(sampled_datasets, browser)
display(recslam_browser.widget)

Sampling frames:   0%|          | 0/666 [00:00<?, ?it/s]

In [ ]:
# ib = ImageBrowser(list(recslam_browser.frames))
# ib.browse()

2023-08-30 13:49:02.602 python[26031:13726256] +[CATransaction synchronize] called within transaction


File /Users/arseniy/Projects/dev/ds_contrib/nbs/core/test.csv saved


In [ ]:
# downloads_dir.cleanup()